## 연도 지정

In [81]:
start_year = 2019
end_year   = 2024
end_date_str = "2024-10-31"

In [82]:
import os
from mysql import connector
from dotenv import load_dotenv
import pandas as pd

# 환경변수 로드
load_dotenv()

NAME = os.getenv("DB_NAME")
USER = os.getenv("DB_USER")
PASSWORD = os.getenv("DB_PASSWORD")
IP = os.getenv("DB_IP")
PORT = os.getenv("DB_PORT")

class Database:
    _connection = None  # 클래스 변수로 연결 상태 관리

    @staticmethod
    def get_connection():
        if Database._connection is None or not Database._connection.is_connected():
            Database._connection = connector.connect(
                database = NAME,
                user = USER,
                password = PASSWORD,
                host = IP,
                port = PORT
            )
        return Database._connection

try:
    # 1) DB 연결
    conn = Database.get_connection()
    cursor = conn.cursor()

    years_range = range(start_year, end_year + 1)
    rc_sum_str = " + ".join([f"SUM(rc.`{year}`)" for year in years_range])
    irc_sum_str = " + ".join([f"SUM(irc.`{year}`)" for year in years_range])

    cursor.execute(f"""
        SELECT 
            book.ID,
            registration_year,
            registration_month,
            get_course,
            DDC,
            title,
            publication_year,
            location,
            duration,
            COALESCE({rc_sum_str}, 0) AS total_rent,
            COALESCE({irc_sum_str}, 0) AS rent_count,
            ID_count
        FROM book
        LEFT JOIN recent_rent       ON book.ID = recent_rent.ID
        LEFT JOIN rent_count AS rc        ON book.ID = rc.ID
        LEFT JOIN ISBN_rent_count AS irc   ON book.ISBN = irc.ISBN
        GROUP BY book.ID, registration_year, registration_month,
                 get_course, DDC, publication_year, location, duration
        ORDER BY book.ID
    """)
    rows_base = cursor.fetchall()
    cursor.close()

except connector.Error as e:
    print(f"Error: {e}")

df = pd.DataFrame(rows_base, columns=[
        'ID', '등록연도', '등록월', '수서방법',
        '분류코드', '제목', '출판연도', '소장위치', '최근대출',
        '총 대출 횟수', 'rent_count', 'book_count'
    ])

In [83]:
import pandas as pd

class Database:
    _connection = None  # 클래스 변수로 연결 상태 관리

    @staticmethod
    def get_connection():
        if Database._connection is None or not Database._connection.is_connected():
            Database._connection = connector.connect(
                database = NAME,
                user = USER,
                password = PASSWORD,
                host = IP,
                port = PORT
            )
        return Database._connection

try:
    conn = Database.get_connection()
    cursor = conn.cursor()

    years_range = range(start_year, end_year + 1)
    sum_str = " + ".join([f"SUM(`{year}`)" for year in years_range])

    cursor.execute(f"SELECT title, ID_count, COALESCE({sum_str}, 0) AS rent_count FROM None_ISBN_rent_count GROUP BY title;")
    
    rows = cursor.fetchall()
    
    cursor.close()
except connector.Error as e:
    print(f"Error: {e}")

None_ISBN_df = pd.DataFrame(rows, columns=['제목', 'book_count', 'rent_count'])

None_ISBN_df = None_ISBN_df.astype(object)
None_ISBN_df['rent_count'] = None_ISBN_df['rent_count'].astype(float)
None_ISBN_df['book_count'] = None_ISBN_df['book_count'].astype(float)

None_ISBN_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6966 entries, 0 to 6965
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   제목          6966 non-null   object 
 1   book_count  6966 non-null   float64
 2   rent_count  6966 non-null   float64
dtypes: float64(2), object(1)
memory usage: 163.4+ KB


In [84]:
df_merged = pd.merge(df, None_ISBN_df, on='제목', how='left')

In [85]:
# → 왼쪽이 NaN인 경우만 오른쪽으로 채우려면:
df_merged['rent_count_x'] = df_merged['rent_count_x'].fillna(df_merged['rent_count_y'])
df_merged['book_count_x'] = df_merged['book_count_x'].fillna(df_merged['book_count_y'])

# 오른쪽 컬럼은 더 이상 필요 없으면 제거
df_merged.drop('rent_count_y', axis=1, inplace=True)
df_merged.drop('book_count_y', axis=1, inplace=True)

# 필요하면 이름을 다시 'val'로 원상복구
df_merged.rename(columns={'rent_count_x': 'rent_count', 'book_count_x': 'book_count'}, inplace=True)

df_merged.drop(columns=['제목'], inplace=True)

In [86]:
# 1. ID 처리: '_' 기준으로 분리, 뒤쪽 값만 사용 후 int로 변환
df_merged['ID'] = df_merged['ID'].str.split('_').str[-1].astype(int)

# 2. 수서 방법을 category로 변환
df_merged['수서방법'] = df_merged['수서방법'].astype('category')

# 3. 분류코드를 float로 변환
df_merged['분류코드'] = df_merged['분류코드'].astype(float)

# 4. 출판연도를 int로 변환
df_merged['출판연도'] = df_merged['출판연도'].astype(int)

# 5. 소장위치를 category로 변환
df_merged['소장위치'] = df_merged['소장위치'].astype('category')

# 6. 최근대출을 int로 변환하고 NaN 값을 7305로 설정
df_merged['최근대출'] = df_merged['최근대출'].fillna(7305).astype(int)
df_merged['rent_count'] = df_merged['rent_count'].fillna(0).astype(int)
df_merged['book_count'] = df_merged['book_count'].fillna(0).astype(int)


# 7. 총 대출 횟수, rent_count, book_count를 int로 변환
df_merged['총 대출 횟수'] = df_merged['총 대출 횟수'].astype(int)

# 결과 확인
df_onehot = pd.get_dummies(df_merged)
df_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299315 entries, 0 to 299314
Data columns (total 17 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ID           299315 non-null  int64  
 1   등록연도         299315 non-null  int64  
 2   등록월          299315 non-null  int64  
 3   분류코드         299315 non-null  float64
 4   출판연도         299315 non-null  int64  
 5   최근대출         299315 non-null  int64  
 6   총 대출 횟수      299315 non-null  int64  
 7   rent_count   299315 non-null  int64  
 8   book_count   299315 non-null  int64  
 9   수서방법_기타      299315 non-null  bool   
 10  수서방법_사서선정    299315 non-null  bool   
 11  수서방법_수서정보없음  299315 non-null  bool   
 12  수서방법_수업지정    299315 non-null  bool   
 13  수서방법_이용자희망   299315 non-null  bool   
 14  수서방법_학과신청    299315 non-null  bool   
 15  소장위치_4층인문    299315 non-null  bool   
 16  소장위치_보존서고    299315 non-null  bool   
dtypes: bool(8), float64(1), int64(8)
memory usage: 22.8 MB


In [87]:
y_data = df_onehot['rent_count'] / df_onehot['book_count']

y_data = y_data.fillna(0)
y_data.info()

<class 'pandas.core.series.Series'>
RangeIndex: 299315 entries, 0 to 299314
Series name: None
Non-Null Count   Dtype  
--------------   -----  
299315 non-null  float64
dtypes: float64(1)
memory usage: 2.3 MB


In [88]:
df_onehot.drop(columns=['rent_count', 'book_count'], inplace=True)

df_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299315 entries, 0 to 299314
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ID           299315 non-null  int64  
 1   등록연도         299315 non-null  int64  
 2   등록월          299315 non-null  int64  
 3   분류코드         299315 non-null  float64
 4   출판연도         299315 non-null  int64  
 5   최근대출         299315 non-null  int64  
 6   총 대출 횟수      299315 non-null  int64  
 7   수서방법_기타      299315 non-null  bool   
 8   수서방법_사서선정    299315 non-null  bool   
 9   수서방법_수서정보없음  299315 non-null  bool   
 10  수서방법_수업지정    299315 non-null  bool   
 11  수서방법_이용자희망   299315 non-null  bool   
 12  수서방법_학과신청    299315 non-null  bool   
 13  소장위치_4층인문    299315 non-null  bool   
 14  소장위치_보존서고    299315 non-null  bool   
dtypes: bool(8), float64(1), int64(6)
memory usage: 18.3 MB


In [89]:
import datetime
start_date = datetime.date(start_year, 1, 1)           # 2019-01-01
year_, month_, day_ = map(int, end_date_str.split('-'))  # 2024, 10, 31
end_date = datetime.date(year_, month_, day_)
diff_days = (end_date - start_date).days  # 2019-01-01 ~ 2024-10-31까지 일수

In [90]:
df_onehot.loc[df_onehot['최근대출'] > diff_days, '최근대출'] = diff_days

In [91]:
# 모듈 불러오기
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [92]:
# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(df_onehot, y_data, test_size=0.3, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(209520, 15)
(89795, 15)
(209520,)
(89795,)


In [93]:
# 모델 불러오기
loaded_booster = xgb.Booster()
loaded_booster.load_model("xgb_model_v1.json")

In [94]:
# 예측
dtest = xgb.DMatrix(df_onehot)
y_pred = loaded_booster.predict(dtest)

In [95]:
rmse = np.sqrt(mean_squared_error(y_data, y_pred))
r2 = r2_score(y_data, y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"R^2 : {r2:.4f}")

RMSE: 1.8295
R^2 : -0.4941


In [96]:
df_y_result = pd.DataFrame({
    "Prediction": y_pred,
    "Actual": y_data
}, index=y_data.index)

In [97]:
display(df_y_result[132478:132500])

,Prediction,Actual
132478,0.294272,0.0
132479,0.285197,0.0
132480,0.285197,0.0
132481,1.004040,1.0
132482,0.336405,0.0
132483,0.934480,1.0
132484,0.890903,0.0
132485,0.979390,1.0
132486,1.229976,1.0
132487,0.350700,0.0


In [98]:
df_onehot[132485:132492]

,ID,등록연도,등록월,분류코드,출판연도,최근대출,총 대출 횟수,수서방법_기타,수서방법_사서선정,수서방법_수서정보없음,수서방법_수업지정,수서방법_이용자희망,수서방법_학과신청,소장위치_4층인문,소장위치_보존서고
132485,132752,2018,7,230.0,2018,1337,1,False,True,False,False,False,False,True,False
132486,132753,2018,7,242.5,2018,675,1,False,True,False,False,False,False,True,False
132487,132754,2018,7,248.4,2018,2130,0,False,True,False,False,False,False,True,False
132488,132755,2018,7,230.0,2018,2130,0,False,True,False,False,False,False,True,False
132489,132756,2018,7,251.0,2018,2130,0,False,True,False,False,False,False,True,False
132490,132757,2018,7,234.0,2017,2130,0,False,True,False,False,False,False,True,False
132491,132758,2018,7,230.0,2018,2130,0,False,True,False,False,False,False,True,False
